In [1]:
!pip install jsonlines

In [2]:
import pandas as pd
import jsonlines
from tqdm import tqdm
import datetime
from glob import glob
import gc

In [3]:
path = '../data/'

## LB Data

In [5]:
!rm -rf $path/train/interim
!mkdir -p $path/train/interim

In [6]:
data_dict = {'session': [], 'aid': [], 'ts': [], 'type': []}

with jsonlines.open(path + '/test.jsonl') as reader:
    for result in tqdm(reader):   
        for event in result['events']:     
            data_dict['session'].append(result['session'])
            data_dict['aid'].append(event['aid'])
            data_dict['ts'].append(event['ts'])
            data_dict['type'].append(event['type'])

1671803it [00:30, 55334.07it/s]


In [7]:
data = pd.DataFrame.from_dict(data_dict)

In [8]:
data.shape, data['session'].drop_duplicates().shape

((6928123, 4), (1671803,))

In [9]:
data.to_parquet(path + '/test.parquet')

In [10]:
del data
gc.collect()

0

In [11]:
chunksize = 100_000
chunks = pd.read_json(path + '/train.jsonl', lines=True, chunksize=chunksize)

for e, chunk in enumerate(tqdm(chunks)):
    data_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }
    
    for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
        for event in events:
            data_dict['session'].append(session)
            data_dict['aid'].append(event['aid'])
            data_dict['ts'].append(event['ts'])
            data_dict['type'].append(event['type'])
            
    df = pd.DataFrame(data_dict)
    df.to_parquet(path + "/train/interim/train_" + str(e) + ".parquet")
    del df
    gc.collect()

129it [28:36, 13.31s/it]


In [12]:
!du -sh $path/lb/interim

du: cannot access '../../data//lb/interim': No such file or directory


## CV Data

In [13]:
path = '../../preprocess/'

In [14]:
!rm -rf $path/train/interim
!mkdir -p $path/train/interim

In [15]:
data_dict = {'session': [], 'aid': [], 'ts': [], 'type': []}

with jsonlines.open(path + 'test_sessions.jsonl') as reader:
    for result in tqdm(reader):   
        for event in result['events']:     
            data_dict['session'].append(result['session'])
            data_dict['aid'].append(event['aid'])
            data_dict['ts'].append(event['ts'])
            data_dict['type'].append(event['type'])

1783737it [00:33, 52472.97it/s]


In [16]:
data = pd.DataFrame.from_dict(data_dict)

In [17]:
data.to_parquet(path + '/test.parquet')

In [18]:
data_dict = {'session': [], 'aid': [], 'ts': [], 'type': []}

with jsonlines.open(path + '/test_sessions_full.jsonl') as reader:
    for result in tqdm(reader):   
        for event in result['events']:     
            data_dict['session'].append(result['session'])
            data_dict['aid'].append(event['aid'])
            data_dict['ts'].append(event['ts'])
            data_dict['type'].append(event['type'])

1783737it [00:54, 32761.99it/s]


In [19]:
data = pd.DataFrame.from_dict(data_dict)

In [20]:
data.to_parquet(path + '/test_full.parquet')

In [21]:
chunksize = 100_000
chunks = pd.read_json(path + '/train_sessions.jsonl', lines=True, chunksize=chunksize)

for e, chunk in enumerate(tqdm(chunks)):
    data_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }
    
    for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
        for event in events:
            data_dict['session'].append(session)
            data_dict['aid'].append(event['aid'])
            data_dict['ts'].append(event['ts'])
            data_dict['type'].append(event['type'])
            
    df = pd.DataFrame(data_dict)
    df.to_parquet(path + "/train/interim/train_" + str(e) + ".parquet")
    del df
    gc.collect()

106it [22:05, 12.50s/it]


In [22]:
data_dict = {'session': [], 'aid': [], 'type': []}

with jsonlines.open(path + '/test_labels.jsonl') as reader:
    for result in tqdm(reader):   
        for event in result['labels'].keys():
            aids = result['labels'][event]
            if not isinstance(aids, list):
                aids = [aids]
            for aid in aids:
                data_dict['session'].append(result['session'])
                data_dict['aid'].append(aid)
                data_dict['type'].append(event)

1783737it [00:17, 103185.99it/s]


In [23]:
data = pd.DataFrame.from_dict(data_dict)

In [24]:
data.to_parquet(path + '/test_labels.parquet')